In [ ]:
import pickle
import numpy as np
import torch
from helper_functions import *

from torch.utils.data import DataLoader,Dataset
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
import torch.nn as nn

import glob
from pathlib import Path
import random

from Network_Training_functions import sort_data,normalize,unnormalize,FE_dataset,MLP,save_net_results
from scipy.io import savemat

In [25]:
# Extract Results from training and Test
train_set,input_vars,output_vars = sort_data('ML_Folder/*.mat',shuffle = False)
valid_set,_,_ = sort_data('ML_Folder/Test/*.mat',shuffle = False)

16
crush_W_0-1067_T_0-001533_V_8-0.mat
crush_W_0-1133_T_0-0018_V_8-0.mat
crush_W_0-1267_T_0-001333_V_8-0.mat
crush_W_0-12_T_0-001067_V_8-0.mat
crush_W_0-1333_T_0-002_V_8-0.mat
crush_W_0-1467_T_0-0012_V_8-0.mat
crush_W_0-14_T_0-001733_V_8-0.mat
crush_W_0-1533_T_0-001467_V_8-0.mat
crush_W_0-1667_T_0-001_V_8-0.mat
crush_W_0-16_T_0-001867_V_8-0.mat
crush_W_0-1733_T_0-001667_V_8-0.mat
crush_W_0-1867_T_0-001933_V_8-0.mat
crush_W_0-18_T_0-0014_V_8-0.mat
crush_W_0-1933_T_0-001133_V_8-0.mat
crush_W_0-1_T_0-001267_V_8-0.mat
crush_W_0-2_T_0-0016_V_8-0.mat
4
crush_W_0-133_T_0-001_V_8-0.mat
crush_W_0-167_T_0-002_V_8-0.mat
crush_W_0-1_T_0-00167_V_8-0.mat
crush_W_0-2_T_0-00133_V_8-0.mat


In [26]:
input_vars

['W', 'T', 'V', 'x', 'y', 'z', 't']

In [27]:
# For this example we eliminate the velocity paramter (parameter 2)
train_set[0] =  train_set[0][:,[0,1,3,4,5,6]]
valid_set[0] =  valid_set[0][:,[0,1,3,4,5,6]]


In [28]:
#Scale Data to [0,1]
out_weights = torch.tensor([0.4,3e-2,3e-2])
in_weights = torch.abs(train_set[0]).max(0)[0]

train_set[0],x1,x2 = normalize(train_set[0])
train_set[1],y1,y2 =  normalize(train_set[1])
valid_set[0],_,_ =  normalize(valid_set[0],x1,x2)

In [29]:
valid_set[0].shape, valid_set[1].shape,train_set[0].shape

(torch.Size([255192, 6]), torch.Size([255192, 3]), torch.Size([1025852, 6]))

In [30]:
train_dataset = FE_dataset(*train_set)
valid_dataset = FE_dataset(*valid_set)
train_dataset.cuda()
valid_dataset.cuda()


train_DL = DataLoader(train_dataset,batch_size=256,shuffle= True,num_workers = 0)
valid_DL = DataLoader(valid_dataset,batch_size=len(valid_dataset),shuffle= False,num_workers = 0)

In [31]:
net = MLP(6,3,256,3)
net = net.cuda()
optimizer = Adam(net.parameters(),lr=1e-3)
lr_scheduler = ExponentialLR(optimizer,gamma = 0.95)

In [32]:

y1,y2 = y1.cuda(),y2.cuda()
out_weights = out_weights.cuda()
in_weights = in_weights.cuda()
optimizer.zero_grad()
for i in range(200):
    net.train()
    train_loss = 0
    train_norm = 0
    for x,y in train_DL:
        
        out = net(x).squeeze()

        train_error =(out-y).pow(2)
        loss = train_error.mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            train_loss += (train_error.sum())
            train_norm += (y.pow(2).sum())
        
    if i % 3 == 0:
        net.eval() 
        train_l2 = torch.sqrt(train_loss/train_norm)
        train_loss = train_loss/len(train_dataset)
        print(len(train_dataset))
        error = 0
        y_norm = 0
        loss = 0
        with torch.no_grad():
            for x,y in (valid_DL):

                out = unnormalize(net(x),y1,y2).squeeze()
                out = out
                err = (out-y).pow(2)
                error += (err.sum())
                y_norm += (y.pow(2).sum())
                
            l2_error = torch.sqrt(error/y_norm)
            loss = error/len(valid_dataset)
            print(f'Epoch {i} LR {float(lr_scheduler.get_last_lr()[0]):.3E} Valid l2_error = {float(l2_error):.3E}, Train L2 Error: {float(train_l2):.3E} Train Loss {float(train_loss):.3E}')
            node_label = err.argmax(0)[0]
            print(node_label)
            print(f'Worst Elements {node_label}, Net Value: {out[node_label]}, True Value {y[node_label]} at Point {[float(xx) for xx in x[node_label]]}')

    if (i+1) % 100 == 0:
        lr_scheduler.step()

1025852
Epoch 0 LR 1.000E-03 Valid l2_error = 1.452E-01, Train L2 Error: 8.325E-02 Train Loss 6.378E-03
tensor(160982, device='cuda:0')
Worst Elements 160982, Net Value: tensor([ 0.0039, -0.0078,  0.0032], device='cuda:0'), True Value tensor([-0.0271, -0.0044,  0.0006], device='cuda:0') at Point [0.0, 0.6699999570846558, 1.0, 0.24998360872268677, 0.38636642694473267, 0.5333525538444519]
1025852
Epoch 3 LR 1.000E-03 Valid l2_error = 9.718E-02, Train L2 Error: 3.202E-02 Train Loss 9.435E-04
tensor(161290, device='cuda:0')
Worst Elements 161290, Net Value: tensor([ 0.0103, -0.0042, -0.0006], device='cuda:0'), True Value tensor([-0.0163,  0.0040,  0.0004], device='cuda:0') at Point [0.0, 0.6699999570846558, 0.9777777791023254, 0.24999144673347473, 0.43181928992271423, 0.5333525538444519]
1025852
Epoch 6 LR 1.000E-03 Valid l2_error = 7.610E-02, Train L2 Error: 2.106E-02 Train Loss 4.080E-04
tensor(122512, device='cuda:0')
Worst Elements 122512, Net Value: tensor([ 0.0586, -0.0013, -0.0107],

KeyboardInterrupt: 

In [33]:
torch.save(net.state_dict(),'network.pth')

In [ ]:
y1,y2 = y1.cpu(),y2.cpu()

ps = glob.glob('ML_Folder/Test/*.mat')
os.makedirs('Test',exist_ok=True)
for p in ps:
    file = Path(p)
    print(file.name)    
    data = loaddata(file)

    to_export = save_net_results(data,net)
    to_export_compact = convert_tabular_to_compact(to_export)
    savemat(f'Test/{file.stem}.mat',to_export)


crush_W_0-133_T_0-001_V_8-0.mat
crush_W_0-167_T_0-002_V_8-0.mat
crush_W_0-1_T_0-00167_V_8-0.mat
crush_W_0-2_T_0-00133_V_8-0.mat
